In [18]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
from langfuse import get_client
from langfuse.langchain import CallbackHandler
 
# Initialize Langfuse client
langfuse = get_client()
 
# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()

In [20]:
from typing_extensions import TypedDict

class WheatherDict(TypedDict):
    """Wheather information"""
    location: str
    weather: str


In [ ]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_ollama import ChatOllama
 
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the whether at a location."""
    return f"It's sunny in {location}."

@tool
def get_hometown(name: str) -> str:
    """Get the hometown of a person."""
    return f"{name}'s hometown is Mount Vernon, New York."

model = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    thinking_level="low",  # For faster, lower-latency responses
    include_thoughts=True, 
)
 
# model = ChatOllama(
#     model="qwen3:8b"
# )

agent = create_agent(
    model=model,
    tools=[get_weather, get_hometown],
    system_prompt="You are a helpful assistant who inform the whether of the user location.",
    response_format=WheatherDict,
)
 


In [22]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "How's weather in denzel washington's hometown?"}]},
    config={"callbacks": [langfuse_handler]}
)

{'messages': [HumanMessage(content="How's weather in denzel washington's hometown?", additional_kwargs={}, response_metadata={}, id='623e51bc-3c2b-41a9-883e-91d7547f5207'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2026-02-02T07:03:29.468053Z', 'done': True, 'done_reason': 'stop', 'total_duration': 36148705583, 'load_duration': 6099391291, 'prompt_eval_count': 239, 'prompt_eval_duration': 2597088250, 'eval_count': 288, 'eval_duration': 27094461075, 'logprobs': None, 'model_name': 'qwen3:8b', 'model_provider': 'ollama'}, id='lc_run--019c1d29-5285-7e01-9fb1-13c61e33eb2d-0', tool_calls=[{'name': 'get_hometown', 'args': {'name': 'Denzel Washington'}, 'id': 'c8bd397c-f0ed-4486-84a9-47c1450cc015', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 239, 'output_tokens': 288, 'total_tokens': 527}),
  ToolMessage(content="Denzel Washington's hometown is Mount Vernon, New York.", name='get_hometown', id='e4c308